In [2]:
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [3]:
default_size=224
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'


model =  Sequential()
model.add(Convolution2D(64, 3, 3, input_shape=(default_size, default_size, 3), activation='relu', border_mode='same', name='block1_conv1'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

In [4]:
f = h5py.File(weights_path)
for k, layer_name in enumerate(f.attrs[u'layer_names']):
    if k == len(model.layers):
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f[layer_name]
    weights = [g[p] for p in g.keys()]
    model.layers[k].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [5]:
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 1600

In [6]:
np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3).shape

(1, 1, 3)

In [7]:

datagen = ImageDataGenerator(rescale=1., featurewise_center=True)
datagen.mean=np.array([103.939, 116.779, 123.68],dtype=np.float32).reshape(1,1,3)

generator = datagen.flow_from_directory(
            train_data_dir,
            target_size=(default_size, default_size),
            batch_size=32,
            class_mode=None,
            shuffle=False)
bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(default_size, default_size),
        batch_size=32,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

Found 2000 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


In [8]:
nb_epoch = 20

train_data = np.load(open('bottleneck_features_train.npy'))
train_labels = np.array([0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

top_model = Sequential()
top_model.add(Flatten(input_shape=train_data.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

top_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

top_model.fit(train_data, train_labels,
          nb_epoch=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels),
          verbose=2)

Train on 2000 samples, validate on 1600 samples
Epoch 1/20
1s - loss: 3.0085 - acc: 0.8045 - val_loss: 1.6272 - val_acc: 0.8925
Epoch 2/20
1s - loss: 2.0824 - acc: 0.8670 - val_loss: 1.0252 - val_acc: 0.9344
Epoch 3/20
1s - loss: 1.2701 - acc: 0.9185 - val_loss: 2.1625 - val_acc: 0.8606
Epoch 4/20
1s - loss: 1.2206 - acc: 0.9210 - val_loss: 0.7136 - val_acc: 0.9531
Epoch 5/20
1s - loss: 0.8449 - acc: 0.9460 - val_loss: 0.7831 - val_acc: 0.9506
Epoch 6/20
1s - loss: 0.9707 - acc: 0.9390 - val_loss: 0.7501 - val_acc: 0.9519
Epoch 7/20
1s - loss: 0.8775 - acc: 0.9435 - val_loss: 1.0532 - val_acc: 0.9325
Epoch 8/20
1s - loss: 0.7405 - acc: 0.9530 - val_loss: 0.6965 - val_acc: 0.9550
Epoch 9/20
1s - loss: 0.7218 - acc: 0.9545 - val_loss: 0.7451 - val_acc: 0.9519
Epoch 10/20
1s - loss: 0.6007 - acc: 0.9615 - val_loss: 0.5356 - val_acc: 0.9656
Epoch 11/20
1s - loss: 0.6065 - acc: 0.9610 - val_loss: 0.6442 - val_acc: 0.9594
Epoch 12/20
1s - loss: 0.4973 - acc: 0.9680 - val_loss: 0.8060 - val_a

In [9]:
top_model_weights_path = 'bottleneck_fc_model.h5'

In [10]:
top_model.save_weights(top_model_weights_path)